<a href="https://colab.research.google.com/github/ishika-b/neural-networks/blob/main/nnfl_assn2_q7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import scipy.io
from scipy.io import loadmat
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import  preprocessing
from sklearn.preprocessing import StandardScaler
normalizer = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
np.random.seed(0)
lmat = loadmat('/content/drive/MyDrive/nnfl/nnfl2/data5.mat')
data = lmat['x']
df = pd.DataFrame(data)
data = np.array(df)

x = data [:,0:72]
y = data[:,72]
m = x.shape[0]

#normalize
means = np.mean(x,axis = 0)
std_devs = np.std(x,axis = 0)
x = (x - means)/std_devs

y = y.reshape(m,1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)

m,n = x_train.shape
m_test, n_test = x_test.shape
m_val, n_val = x_val.shape

y_tr = np.zeros((m,2), dtype=int)
for i in range(m):
  val = int(y_train[i][0])
  y_tr[i][val] = 1

dim = [72,3000,5000,2]

lambd = 0.001

x_train = np.insert(x_train, 0, np.ones(m), axis = 1) #(1503,73)
random_mat = np.random.standard_normal(size=(dim[1], dim[0]+1)) #(3000,73) hidden layer weights 
H1 = np.tanh(np.dot(random_mat, x_train.T)) #(3000,1503) hidden layer values 
w1_temp = np.dot(H1,H1.T)+ np.eye(dim[1])/lambd #(HTH +1/lam*I) 
w1 = np.dot(H1.T, np.linalg.inv(w1_temp)) #multiplied by H.T
W1 = np.dot(w1.T,x_train) #(3000,73) #weight matrix
H1 = H1.T #1503,3000

lambd = 0.0001

H1 = np.insert(H1, 0, np.ones(m), axis = 1) #(1503,3001)
random_mat = np.random.standard_normal(size=(dim[2], dim[1]+1)) #(5000,3001)
H2 = np.tanh(np.dot(random_mat, H1.T)) #(5000,3001)*(3001,1503)
w2_temp = np.dot(H2,H2.T)+ np.eye(dim[2])/lambd #(5000,1503)(1503,5000)
w2 = np.dot(H2.T, np.linalg.inv(w2_temp)) #(1503,5000)*(5000,5000)
W2 = np.dot(w2.T, H1) #(5000,1503)*(1503,3001)=(5000,3001)
H2 = H2.T#1503,5000

In [ ]:
H2 = np.insert(H2, 0, np.ones(m), axis = 1) #1503,5001
w3_temp = np.dot(H2.T,H2)  +  np.eye(dim[2]+1)/lambd
w3 = np.dot(H2, np.linalg.pinv(w3_temp))
W3 = (np.dot(w3.T,y_tr)).T #(2,5001)

print(H2.shape,W3.shape)

(1503, 5001) (2, 5001)


In [ ]:
y_pred = np.dot(H2,W3.T)
preds = np.argmax(y_pred, axis = 1)
print('overall training accuracy:', accuracy_score(y_train, preds))

overall training accuracy: 0.9707252162341983


In [ ]:
#VALIDATION 
x_val = np.insert(x_val,0,np.ones(m_val),axis=1)
H1_val = np.tanh(np.dot(W1,x_val.T)).T
H1_val = np.insert(H1_val,0,np.ones(m_val),axis=1) 
H2_val = np.tanh(np.dot(W2,H1_val.T)).T #Lxm
H2_val = np.insert(H2_val,0,np.ones(m_val),axis=1) 
y_pred_val = np.dot(W3,H2_val.T).T
preds = np.argmax(y_pred_val, axis = 1)
print('overall validation accuracy:', accuracy_score(y_val, preds))  

overall validation accuracy: 0.827906976744186


In [ ]:
cm = confusion_matrix(y_val, preds)
print("validation class 0 accuracy:", cm[0,0]/(cm[0,0]+cm[0,1]))
print("validation class 1 accuracy:", cm[1,1]/(cm[1,1]+cm[1,0]))

validation class 0 accuracy: 0.7830188679245284
validation class 1 accuracy: 0.8715596330275229


In [ ]:
#TESTING
x_test = np.insert(x_test,0,np.ones(m_test),axis=1)
H1_test = np.tanh(np.dot(W1,x_test.T)).T
H1_test = np.insert(H1_test,0,np.ones(m_test),axis=1) 
H2_test = np.tanh(np.dot(W2,H1_test.T)).T #Lxm
H2_test = np.insert(H2_test,0,np.ones(m_test),axis=1) 
y_pred_test = np.dot(W3,H2_test.T).T
preds = np.argmax(y_pred_test, axis = 1)
print('overall testing accuracy:', accuracy_score(y_test, preds))  

overall testing accuracy: 0.8232558139534883


In [ ]:
cm = confusion_matrix(y_test, preds)
print("testing class 0 accuracy:", cm[0,0]/(cm[0,0]+cm[0,1]))
print("testing class 1 accuracy:", cm[1,1]/(cm[1,1]+cm[1,0]))

testing class 0 accuracy: 0.7924528301886793
testing class 1 accuracy: 0.8532110091743119
